# загрузка файлов

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1fqEBaIkTTjxM-4bZ4AK0WulM087ofkQw'})

In [ ]:
download.GetContentFile('bar.zip')

In [ ]:
!unzip bar.zip

Archive:  bar.zip
  inflating: spb/description/spb_market.json  
  inflating: spb/description/spb_bars_and_clubs.json  
  inflating: spb/description/spb_fast_snacks.json  
  inflating: spb/description/spb_deserts.json  
  inflating: spb/reviews/spb_market_reviews.json  
  inflating: spb/description/spb_bakery.json  
  inflating: spb/description/spb_coffee_and_tea.json  
  inflating: spb/reviews/spb_coffee_and_tea_reviews.json  
  inflating: spb/reviews/spb_deserts_reviews.json  
  inflating: spb/reviews/spb_bars_and_clubs_reviews.json  
  inflating: spb/reviews/spb_fast_snacks_reviews.json  
  inflating: spb/reviews/spb_bakery_reviews.json  
  inflating: spb/description/spb_restaurants.json  
  inflating: spb/reviews/spb_restaurants_reviews.json  


# препроцессинг

In [ ]:
import pandas as pd
import json
import requests

In [ ]:
with open('/content/spb/description/spb_bars_and_clubs.json') as f:
    data = pd.DataFrame(json.loads(line) for line in f)

In [ ]:
api_key = ''

In [ ]:
def create_urls (adr, api_key=''):
  base_request = 'https://geocode-maps.yandex.ru/1.x/?apikey={api_key}&format=json&geocode={adr}'
  return base_request.format(adr=adr, api_key=api_key)

In [ ]:
def get_urls(addrs):
  urls = []
  for addr in addrs:
    urls.append(create_urls(addr.replace(' ', '+')))
  return urls

In [ ]:
def get_coords(urls):
  coords = []
  #addr = []
  for url in urls:
    r = requests.get(url)
    coords.append((
        float(r.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()[0]),
        float(r.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()[1])
        ))
    #addr.append(r.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['metaDataProperty']['GeocoderMetaData']['text'])
  return coords

In [ ]:
#addr_new = pd.read_csv('addr.csv')
addr_new = pd.read_excel('addr.xlsx')

In [ ]:
data['address'] = addr_new

In [ ]:
for i in range(1, 4):
  data['f%s'%i] = data['feature_%s_name'%i] + ': ' + data['feature_%s'%i]

In [ ]:
data['start'] = pd.to_datetime(data['schedule_start'] + data['schedule_start_letter']).dt.strftime('%H:%M')

In [ ]:
data['end'] = pd.to_datetime(data['schedule_end'] + data['schedule_end_letter']).dt.strftime('%H:%M')

In [ ]:
data = data.drop(['ctime', 'schedule_start', 'schedule_start_letter', 'schedule_end', 'schedule_end_letter', 
                  'feature_1_name', 'feature_1', 'feature_2_name', 'feature_2', 'feature_3_name', 'feature_3'], axis=1)

In [ ]:
u = get_urls(data['address'])

In [ ]:
data['coords'] = get_coords(u)

In [ ]:
data = data.where(pd.notnull(data), None)

In [ ]:
data.to_csv('df.csv', index=False)